In [1]:
! cd ~

/bin/bash: /home/cosmo/anaconda3/envs/TensorFlow_2.8.3__Python_3.9/lib/libtinfo.so.6: no version information available (required by /bin/bash)


In [2]:
! git clone https://github.com/tensorflow/models.git

/bin/bash: /home/cosmo/anaconda3/envs/TensorFlow_2.8.3__Python_3.9/lib/libtinfo.so.6: no version information available (required by /bin/bash)
正複製到 'models'...
remote: Enumerating objects: 86202, done.
remote: Counting objects: 100% (1801/1801), done.
remote: Compressing objects: 100% (757/757), done.
remote: Total 86202 (delta 1154), reused 1635 (delta 1025), pack-reused 84401
接收物件中: 100% (86202/86202), 598.87 MiB | 17.23 MiB/s, 完成.
處理 delta 中: 100% (61656/61656), 完成.


In [3]:
! pip install contextlib2
import os
new_python_path = (os.environ.get("PYTHONPATH") or '') + ":models/research/slim"
%env PYTHONPATH=$new_python_path

/bin/bash: /home/cosmo/anaconda3/envs/TensorFlow_2.8.3__Python_3.9/lib/libtinfo.so.6: no version information available (required by /bin/bash)
env: PYTHONPATH=:models/research/slim


# 下載數據集
[Visual Wake Words](https://arxiv.org/abs/1906.05721) 數據集包含屬於兩類的圖像：人（標記為 1）和非人（標記為 0），它源自包含80 個類別（例如：貓、狗、雨傘等）的[COCO 數據集](https://cocodataset.org/#explore)。您可以通過運行以下腳本來下載數據集：

In [4]:
! python models/research/slim/download_and_convert_data.py \
--logtostderr \
--dataset_name=visualwakewords \
--dataset_dir=person_detection_dataset \
--foreground_class_of_interest='person' \
--small_object_area_threshold=0.005

/bin/bash: /home/cosmo/anaconda3/envs/TensorFlow_2.8.3__Python_3.9/lib/libtinfo.so.6: no version information available (required by /bin/bash)
2023-06-24 12:48:26.028059: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-06-24 12:48:26.028189: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-06-24 12:48:26.041416: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-06-24 12:48:26.041550: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so re

這將需要幾分鐘（約 20 分鐘或更長時間），因此您可能需要等待一段時間才能繼續下一部分。完成後，您將在 person_detection_dataset/目錄中擁有一組 TFRecord，其中包含標記的圖像信息。

該腳本需要很長時間，因為 COCO 數據集沒有每個圖像的標籤，而是每個圖像都帶有一個帶標籤的邊界框列表。為了創建 Visual WakeWords 數據集，我們循環遍歷每個圖像及其邊界框，如果圖像至少有一個標記為“人”的邊界框，且其面積大於圖像面積的 0.5%，則整個圖像是標記為“人”，否則標記為“非人”。

# 訓練模型

In [6]:
! python models/research/slim/train_image_classifier.py \
    --alsologtostderr \
    --dataset_name=visualwakewords \
    --dataset_dir=person_detection_dataset \
    --dataset_split_name=train \
    --train_image_size=96 \
    --use_grayscale=True \
    --preprocessing_name=mobilenet_v1 \
    --model_name=mobilenet_v1_025 \
    --train_dir=person_detection_train \
    --save_summaries_secs=300 \
    --learning_rate=0.045 \
    --label_smoothing=0.1 \
    --learning_rate_decay_factor=0.98 \
    --num_epochs_per_decay=2.5 \
    --moving_average_decay=0.9999 \
    --batch_size=96 \
    --max_number_of_steps=1000000

/bin/bash: /home/cosmo/anaconda3/envs/TensorFlow_2.8.3__Python_3.9/lib/libtinfo.so.6: no version information available (required by /bin/bash)
Traceback (most recent call last):
  File "/home/cosmo/SideProjects/TensorFlow-and-PyTorch-official-tutorial-2023/TensorFlow/tensorflow-lite/object-detection/person-detection/models/research/slim/train_image_classifier.py", line 24, in <module>
    from tensorflow.contrib import quantize as contrib_quantize
ModuleNotFoundError: No module named 'tensorflow.contrib'


在單 GPU v100 實例上完成所有一百萬步將需要幾天時間，但如果您想儘早進行實驗，您應該能夠在幾個小時後獲得相當準確的模型。